In [1]:
import json
import numpy as np
import tensorflow as tf
import collections
from sklearn.cross_validation import train_test_split

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
with open('ctexts.json','r') as fopen:
    ctexts = json.load(fopen)
    
with open('headlines.json','r') as fopen:
    headlines = json.load(fopen)

In [3]:
import re
from unidecode import unidecode

def textcleaning(string):
    string = unidecode(string).replace('.', '. ').replace(',', ' , ')
    string = re.sub('[^\'"A-Za-z\- ]+', ' ', string)
    string = re.sub(r'[ ]+', ' ', string.lower()).strip()
    return string

In [4]:
%%time
h, c = [], []
for i in range(len(ctexts)):
    try:
        c.append(textcleaning(ctexts[i]))
        h.append(textcleaning(headlines[i]))
    except:
        pass

CPU times: user 528 ms, sys: 0 ns, total: 528 ms
Wall time: 528 ms


In [5]:
def build_dataset(words, n_words):
    count = [['PAD', 0], ['GO', 1], ['EOS', 2], ['UNK', 3]]
    count.extend(collections.Counter(words).most_common(n_words))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [6]:
concat_from = ' '.join(c).split()
vocabulary_size_from = len(list(set(concat_from)))
data_from, count_from, dictionary_from, rev_dictionary_from = build_dataset(concat_from, vocabulary_size_from)
print('vocab from size: %d'%(vocabulary_size_from))
print('Most common words', count_from[4:10])
print('Sample data', data_from[:10], [rev_dictionary_from[i] for i in data_from[:10]])

vocab from size: 49354
Most common words [('the', 98003), ('dot', 79175), ('comma', 74246), ('and', 42548), ('to', 41160), ('of', 38549)]
Sample data [4, 11267, 7, 13170, 655, 12, 504, 8012, 11, 4043] ['the', 'daman', 'and', 'diu', 'administration', 'on', 'wednesday', 'withdrew', 'a', 'circular']


In [7]:
concat_to = ' '.join(h).split()
vocabulary_size_to = len(list(set(concat_to)))
data_to, count_to, dictionary_to, rev_dictionary_to = build_dataset(concat_to, vocabulary_size_to)
print('vocab to size: %d'%(vocabulary_size_to))
print('Most common words', count_to[4:10])
print('Sample data', data_to[:10], [rev_dictionary_to[i] for i in data_to[:10]])

vocab to size: 8330
Most common words [('to', 1388), ('in', 1196), ('comma', 876), ('s', 787), ('for', 733), ('of', 596)]
Sample data [2749, 14, 2750, 2751, 656, 2752, 5, 1623, 657, 2051] ['daman', 'and', 'diu', 'revokes', 'mandatory', 'rakshabandhan', 'in', 'offices', 'order', 'malaika']


In [8]:
for i in range(len(h)):
    h[i] = h[i] + ' EOS'
h[0]

'daman and diu revokes mandatory rakshabandhan in offices order EOS'

In [9]:
GO = dictionary_from['GO']
PAD = dictionary_from['PAD']
EOS = dictionary_from['EOS']
UNK = dictionary_from['UNK']

In [10]:
train_X, test_X, train_Y, test_Y = train_test_split(c, h, test_size = 0.2)

In [11]:
def sent2idx(sent, vocab, UNK=UNK):
    tokens = sent.split()
    oovs = []
    extend_tokens = []
    tokenized = []
    for token in tokens:
        if token not in vocab:
            tokenized.append(UNK)
            if token not in oovs:
                oovs.append(token)
            extend_tokens.append(len(vocab) + oovs.index(token))
        else:
            extend_tokens.append(vocab[token])
            tokenized.append(vocab[token])
    return tokenized, extend_tokens, oovs

def target2idx(sent, oovs, vocab,UNK=UNK):
    tokens = sent.split()
    tokenized = []
    for token in tokens:
        if token not in vocab:
            if token not in oovs:
                tokenized.append(UNK)
            else:
                tokenized.append(len(vocab) + oovs.index(token))
        else:
            tokenized.append(vocab[token])
    return tokenized

In [12]:
import tensorflow as tf
from tensorflow.python.framework import ops, tensor_shape
from tensorflow.contrib.rnn.python.ops.core_rnn_cell import _linear
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
from tensorflow.python.util import nest
from tensorflow.python.ops import init_ops, array_ops, variable_scope
from tensorflow.contrib.seq2seq.python.ops.attention_wrapper import (
    _compute_attention,
)

UNK_ID = 3


class PointerGeneratorGreedyEmbeddingHelper(
    tf.contrib.seq2seq.GreedyEmbeddingHelper
):
    def __init__(self, embedding, start_tokens, end_token):
        self.vocab_size = tf.shape(embedding)[-1]
        super(PointerGeneratorGreedyEmbeddingHelper, self).__init__(
            embedding, start_tokens, end_token
        )

    def sample(self, time, outputs, state, name = None):
        """sample for PointerGeneratorGreedyEmbeddingHelper."""
        del time, state  # unused by sample_fn
        # Outputs are logits, use argmax to get the most probable id
        if not isinstance(outputs, ops.Tensor):
            raise TypeError(
                'Expected outputs to be a single Tensor, got: %s'
                % type(outputs)
            )
        sample_ids = tf.argmax(outputs, axis = -1, output_type = tf.int32)
        return sample_ids

    def next_inputs(self, time, outputs, state, sample_ids, name = None):
        """next_inputs_fn for GreedyEmbeddingHelper."""
        del time, outputs  # unused by next_inputs_fn
        finished = tf.equal(sample_ids, self._end_token)
        all_finished = tf.reduce_all(finished)

        # since we have OOV words, we need change these words to UNK
        condition = tf.less(sample_ids, self.vocab_size)
        sample_ids = tf.where(
            condition, sample_ids, tf.ones_like(sample_ids) * UNK_ID
        )

        next_inputs = tf.cond(
            all_finished,
            # If we're finished, the next_inputs value doesn't matter
            lambda: self._start_inputs,
            lambda: self._embedding_fn(sample_ids),
        )
        return (finished, next_inputs, state)


class PointerGeneratorDecoder(tf.contrib.seq2seq.BasicDecoder):
    """Pointer Generator sampling decoder."""

    def __init__(
        self,
        source_extend_tokens,
        source_oov_words,
        coverage,
        cell,
        helper,
        initial_state,
        output_layer = None,
    ):
        self.source_oov_words = source_oov_words
        self.source_extend_tokens = source_extend_tokens
        self.coverage = coverage
        super(PointerGeneratorDecoder, self).__init__(
            cell, helper, initial_state, output_layer
        )

    @property
    def output_size(self):
        # Return the cell output and the id
        return tf.contrib.seq2seq.BasicDecoderOutput(
            rnn_output = self._rnn_output_size() + self.source_oov_words,
            sample_id = self._helper.sample_ids_shape,
        )

    @property
    def output_dtype(self):
        # Assume the dtype of the cell is the output_size structure
        # containing the input_state's first component's dtype.
        # Return that structure and the sample_ids_dtype from the helper.
        dtype = nest.flatten(self._initial_state)[0].dtype
        return tf.contrib.seq2seq.BasicDecoderOutput(
            nest.map_structure(
                lambda _: dtype, self._rnn_output_size() + self.source_oov_words
            ),
            self._helper.sample_ids_dtype,
        )

    def step(self, time, inputs, state, name = None):
        """Perform a decoding step.
        Args:
        time: scalar `int32` tensor.
        inputs: A (structure of) input tensors.
        state: A (structure of) state tensors and TensorArrays.
        name: Name scope for any created operations.
        Returns:
        `(outputs, next_state, next_inputs, finished)`.
        """
        with ops.name_scope(name, 'PGDecoderStep', (time, inputs, state)):
            cell_outputs, cell_state = self._cell(inputs, state)
            # the first cell state contains attention, which is context
            attention = cell_state[0].attention
            att_cell_state = cell_state[0].cell_state
            alignments = cell_state[0].alignments

            with tf.variable_scope('calculate_pgen'):
                p_gen = _linear([attention, inputs, att_cell_state], 1, True)
                p_gen = tf.sigmoid(p_gen)

            if self._output_layer is not None:
                cell_outputs = self._output_layer(cell_outputs)

            vocab_dist = tf.nn.softmax(cell_outputs) * p_gen

            # z = tf.reduce_sum(alignments,axis=1)
            # z = tf.reduce_sum(tf.cast(tf.less_equal(alignments, 0),tf.int32))
            alignments = alignments * (1 - p_gen)

            # x = tf.reduce_sum(tf.cast(tf.less_equal((1-p_gen), 0),tf.int32))
            # y = tf.reduce_sum(tf.cast(tf.less_equal(alignments[3], 0),tf.int32))

            # this is only for debug
            # alignments2 =  tf.Print(alignments2,[tf.shape(inputs),x,y,alignments[2][9:12]],message="zeros in vocab dist and alignments")

            # since we have OOV words, we need expand the vocab dist
            vocab_size = tf.shape(vocab_dist)[-1]
            extended_vsize = vocab_size + self.source_oov_words
            batch_size = tf.shape(vocab_dist)[0]
            extra_zeros = tf.zeros((batch_size, self.source_oov_words))
            # batch * extend vocab size
            vocab_dists_extended = tf.concat(
                axis = -1, values = [vocab_dist, extra_zeros]
            )
            # vocab_dists_extended = tf.Print(vocab_dists_extended,[tf.shape(vocab_dists_extended),self.source_oov_words],message='vocab_dists_extended size')

            batch_nums = tf.range(0, limit = batch_size)  # shape (batch_size)
            batch_nums = tf.expand_dims(batch_nums, 1)  # shape (batch_size, 1)
            attn_len = tf.shape(self.source_extend_tokens)[
                1
            ]  # number of states we attend over
            batch_nums = tf.tile(
                batch_nums, [1, attn_len]
            )  # shape (batch_size, attn_len)
            indices = tf.stack(
                (batch_nums, self.source_extend_tokens), axis = 2
            )  # shape (batch_size, enc_t, 2)
            shape = [batch_size, extended_vsize]
            attn_dists_projected = tf.scatter_nd(indices, alignments, shape)

            final_dists = attn_dists_projected + vocab_dists_extended
            # final_dists = tf.Print(final_dists,[tf.reduce_sum(tf.cast(tf.less_equal(final_dists[0],0),tf.int32))],message='final dist')
            # note: sample_ids will contains OOV words
            sample_ids = self._helper.sample(
                time = time, outputs = final_dists, state = cell_state
            )

            (finished, next_inputs, next_state) = self._helper.next_inputs(
                time = time,
                outputs = cell_outputs,
                state = cell_state,
                sample_ids = sample_ids,
            )

            outputs = tf.contrib.seq2seq.BasicDecoderOutput(
                final_dists, sample_ids
            )
            return (outputs, next_state, next_inputs, finished)


class PointerGeneratorAttentionWrapper(tf.contrib.seq2seq.AttentionWrapper):
    def __init__(
        self,
        cell,
        attention_mechanism,
        attention_layer_size = None,
        alignment_history = False,
        cell_input_fn = None,
        output_attention = True,
        initial_cell_state = None,
        name = None,
        coverage = False,
    ):
        super(PointerGeneratorAttentionWrapper, self).__init__(
            cell,
            attention_mechanism,
            attention_layer_size,
            alignment_history,
            cell_input_fn,
            output_attention,
            initial_cell_state,
            name,
        )
        self.coverage = coverage

    def zero_state(self, batch_size, dtype):
        with ops.name_scope(
            type(self).__name__ + 'ZeroState', values = [batch_size]
        ):
            if self._initial_cell_state is not None:
                cell_state = self._initial_cell_state
            else:
                cell_state = self._cell.zero_state(batch_size, dtype)
            error_message = (
                'When calling zero_state of AttentionWrapper %s: '
                % self._base_name
                + 'Non-matching batch sizes between the memory '
                '(encoder output) and the requested batch size.  Are you using '
                'the BeamSearchDecoder?  If so, make sure your encoder output has '
                'been tiled to beam_width via tf.contrib.seq2seq.tile_batch, and '
                'the batch_size= argument passed to zero_state is '
                'batch_size * beam_width.'
            )
            with tf.control_dependencies(
                self._batch_size_checks(batch_size, error_message)
            ):
                cell_state = nest.map_structure(
                    lambda s: tf.identity(s, name = 'checked_cell_state'),
                    cell_state,
                )
            return tf.contrib.seq2seq.AttentionWrapperState(
                cell_state = cell_state,
                time = tf.zeros([], dtype = tf.int32),
                attention = _zero_state_tensors(
                    self._attention_layer_size, batch_size, dtype
                ),
                alignments = self._item_or_tuple(
                    attention_mechanism.initial_alignments(batch_size, dtype)
                    for attention_mechanism in self._attention_mechanisms
                ),
                attention_state = self._item_or_tuple(
                    attention_mechanism.initial_state(batch_size, dtype)
                    for attention_mechanism in self._attention_mechanisms
                ),
                # since we need to read the alignment history several times, so we need set clear_after_read to False
                alignment_history = self._item_or_tuple(
                    tf.TensorArray(
                        dtype = dtype,
                        size = 0,
                        clear_after_read = False,
                        dynamic_size = True,
                    )
                    if self._alignment_history
                    else ()
                    for _ in self._attention_mechanisms
                ),
            )

    def call(self, inputs, state):
        if not isinstance(state, tf.contrib.seq2seq.AttentionWrapperState):
            raise TypeError(
                'Expected state to be instance of AttentionWrapperState. '
                'Received type %s instead.' % type(state)
            )

        # Step 1: Calculate the true inputs to the cell based on the
        # previous attention value.
        cell_inputs = self._cell_input_fn(inputs, state.attention)
        cell_state = state.cell_state
        cell_output, next_cell_state = self._cell(cell_inputs, cell_state)

        cell_batch_size = cell_output.shape[0].value or tf.shape(cell_output)[0]
        error_message = (
            'When applying AttentionWrapper %s: ' % self.name
            + 'Non-matching batch sizes between the memory '
            '(encoder output) and the query (decoder output).  Are you using '
            'the BeamSearchDecoder?  You may need to tile your memory input via '
            'the tf.contrib.seq2seq.tile_batch function with argument '
            'multiple=beam_width.'
        )
        with tf.control_dependencies(
            self._batch_size_checks(cell_batch_size, error_message)
        ):
            cell_output = tf.identity(cell_output, name = 'checked_cell_output')

        if self._is_multi:
            previous_alignments = state.alignments
            previous_alignment_history = state.alignment_history
        else:
            previous_alignments = [state.alignments]
            previous_alignment_history = [state.alignment_history]

        all_alignments = []
        all_attentions = []
        all_histories = []

        for i, attention_mechanism in enumerate(self._attention_mechanisms):
            print(attention_mechanism)
            if self.coverage:
                # if we use coverage mode, previous alignments is coverage vector
                # alignment history stack has shape:  decoder time * batch * atten_len
                # convert it to coverage vector
                previous_alignments[i] = tf.cond(
                    previous_alignment_history[i].size() > 0,
                    lambda: tf.reduce_sum(
                        tf.transpose(
                            previous_alignment_history[i].stack(), [1, 2, 0]
                        ),
                        axis = 2,
                    ),
                    lambda: tf.zeros_like(previous_alignments[i]),
                )
            # debug
            # previous_alignments[i] = tf.Print(previous_alignments[i],[previous_alignment_history[i].size(), tf.shape(previous_alignments[i]),previous_alignments[i]],message="atten wrapper:")
            attention, alignments, next_attention_state = _compute_attention(
                attention_mechanism,
                cell_output,
                previous_alignments[i],
                self._attention_layers[i] if self._attention_layers else None,
            )
            alignment_history = (
                previous_alignment_history[i].write(state.time, alignments)
                if self._alignment_history
                else ()
            )

            all_alignments.append(alignments)
            all_histories.append(alignment_history)
            all_attentions.append(attention)

        attention = tf.concat(all_attentions, 1)
        next_state = tf.contrib.seq2seq.AttentionWrapperState(
            time = state.time + 1,
            cell_state = next_cell_state,
            attention = attention,
            alignments = self._item_or_tuple(all_alignments),
            attention_state = self._item_or_tuple(all_alignments),
            alignment_history = self._item_or_tuple(all_histories),
        )

        if self._output_attention:
            return attention, next_state
        else:
            return cell_output, next_state


def _pg_luong_score(query, keys, scale, coverage, coverage_vector):
    depth = query.get_shape()[-1]
    key_units = keys.get_shape()[-1]
    num_units = keys.shape[2].value or tf.shape(keys)[2]
    if depth != key_units:
        raise ValueError(
            'Incompatible or unknown inner dimensions between query and keys.  '
            'Query (%s) has units: %s.  Keys (%s) have units: %s.  '
            "Perhaps you need to set num_units to the keys' dimension (%s)?"
            % (query, depth, keys, key_units, key_units)
        )
    dtype = query.dtype
    query = array_ops.expand_dims(query, 1)
    score = tf.matmul(query, keys, transpose_b = True)
    score = array_ops.squeeze(score, [1])
    if scale:
        g = variable_scope.get_variable(
            'attention_g',
            dtype = dtype,
            initializer = init_ops.ones_initializer,
            shape = (),
        )
        score = g * score
    if coverage:
        print(score, coverage_vector)
        w_c = tf.get_variable('coverage_w', [num_units], dtype = dtype)
        coverage_vector = tf.expand_dims(coverage_vector, -1)
        score = tf.expand_dims(score, -1)
        return tf.reduce_sum(score + coverage_vector, [2])


class PointerGeneratorLuongAttention(tf.contrib.seq2seq.LuongAttention):
    def __init__(
        self,
        num_units,
        memory,
        memory_sequence_length = None,
        probability_fn = None,
        score_mask_value = float('-inf'),
        name = 'PointerGeneratorLuongAttention',
        coverage = False,
        scale = False,
    ):
        super(PointerGeneratorLuongAttention, self).__init__(
            num_units = num_units,
            memory = memory,
            scale = scale,
            memory_sequence_length = memory_sequence_length,
            probability_fn = probability_fn,
            score_mask_value = score_mask_value,
            name = name,
        )
        self.coverage = coverage

    def __call__(self, query, state):
        with tf.variable_scope(
            None, 'pointer_generator_luong_attention', [query]
        ):

            score = _pg_luong_score(
                query, self._keys, self._scale, self.coverage, state
            )
            print(score)
        alignments = self._probability_fn(score, state)
        next_state = alignments
        return alignments, next_state


In [13]:
class Summarization:
    def __init__(self, size_layer, num_layers, embedded_size, 
                 from_dict_size):
        
        def lstm_cell(reuse=False):
            return tf.nn.rnn_cell.GRUCell(size_layer, reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]
        self.source_oov_words = tf.placeholder(tf.int32, shape=[])
        self.source_extend_tokens = tf.placeholder(tf.int32, shape=[None, None])
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        
        condition = tf.less(decoder_input, from_dict_size)
        self.decoder_input = self.Y
        self.decoder_input_length = self.Y_seq_len
        self.predict_count = tf.reduce_sum(self.decoder_input_length)
        
        encoder_embeddings = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        decoder_embeddings = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        encoder_cells = tf.nn.rnn_cell.MultiRNNCell([lstm_cell() for _ in range(num_layers)])
        self.encoder_out, self.encoder_state = tf.nn.dynamic_rnn(cell = encoder_cells, 
                                                                 inputs = encoder_embedded, 
                                                                 sequence_length = self.X_seq_len,
                                                                 dtype = tf.float32)
        self.decode_initial_state = self.encoder_state[-1]
        print(self.decode_initial_state)
        
        atten_mech = PointerGeneratorLuongAttention(
                size_layer, self.encoder_out, memory_sequence_length=self.X_seq_len,
        coverage = True)
        decoder_cells = [lstm_cell() for _ in range(num_layers)]
        decoder_cells[0] = PointerGeneratorAttentionWrapper(
                cell=decoder_cells[0],
                attention_mechanism=atten_mech,
                attention_layer_size=size_layer,
                alignment_history = True,
                coverage = True
            )
        initial_state = [self.decode_initial_state for i in range(num_layers)]
        attention_cell_state = decoder_cells[0].zero_state(
                dtype=tf.float32, batch_size=batch_size)
        initial_state[0] = attention_cell_state.clone(
                cell_state=initial_state[0])
        self.initial_state = tuple(initial_state)
        decoder_cells = tf.contrib.rnn.MultiRNNCell(decoder_cells)
        
        decoded = tf.nn.embedding_lookup(decoder_embeddings, self.decoder_input)
        
        training_helper = tf.contrib.seq2seq.TrainingHelper(
            decoded,
            self.decoder_input_length
        )
        dense_layer = tf.layers.Dense(from_dict_size)
        
        training_decoder = PointerGeneratorDecoder(
            source_extend_tokens = self.source_extend_tokens,
            source_oov_words = self.source_oov_words,
            coverage = True,
            cell=decoder_cells,
            helper=training_helper,
            initial_state=self.initial_state,
            output_layer=dense_layer
        )
        
        maxlen = tf.reduce_max(self.decoder_input_length)
        train_dec_outputs, train_dec_last_state, _ = tf.contrib.seq2seq.dynamic_decode(
            training_decoder,
            output_time_major=False,
            impute_finished=True,
            maximum_iterations=maxlen,
            swap_memory=True)
        logits = train_dec_outputs.rnn_output
        
        masks = tf.sequence_mask(
            self.decoder_input_length, maxlen, 
            dtype=tf.float32)
        
        targets = tf.slice(decoder_input, [0, 0], [-1, maxlen])
        i1, i2 = tf.meshgrid(tf.range(batch_size),
                     tf.range(maxlen), indexing="ij")
        indices = tf.stack((i1,i2,targets),axis=2)
        probs = tf.gather_nd(logits, indices)
        probs = tf.where(tf.less_equal(probs,0),tf.ones_like(probs)*1e-10,probs)
        crossent = -tf.log(probs)
        self.cost = tf.reduce_sum(crossent * masks) / tf.to_float(batch_size)
        alignment_history = train_dec_last_state[0].alignment_history.stack()
        alignment_history = tf.transpose(alignment_history,[1,2,0])
        coverage_loss = tf.minimum(alignment_history,tf.cumsum(alignment_history, axis=2, exclusive=True))
        self.coverage_loss = tf.reduce_sum(coverage_loss / tf.to_float(batch_size))
        self.cost = self.cost + self.coverage_loss
        
        self.optimizer = tf.train.AdamOptimizer().minimize(self.cost)
        
        helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
            embedding=decoder_embeddings,
            start_tokens=tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
            end_token=EOS
        )
        
        inference_decoder = PointerGeneratorDecoder(
            source_extend_tokens = self.source_extend_tokens,
            source_oov_words = self.source_oov_words,
            coverage = True,
            cell=decoder_cells,
            helper=helper,
            initial_state=self.initial_state,
            output_layer=dense_layer
        )
        
        dec_outputs, dec_last_state, _ = tf.contrib.seq2seq.dynamic_decode(
            inference_decoder,
            output_time_major=False,
            maximum_iterations=tf.reduce_max(self.X_seq_len),
            swap_memory=True)
        
        self.beam_predictions = dec_outputs.sample_id
        print(self.beam_predictions)
        
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        y_t = tf.argmax(logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [14]:
size_layer = 128
num_layers = 2
embedded_size = 128
batch_size = 8

In [15]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Summarization(size_layer, num_layers, embedded_size, len(dictionary_from))
sess.run(tf.global_variables_initializer())

Tensor("rnn/while/Exit_4:0", shape=(?, 128), dtype=float32)
Tensor("decoder/while/PGDecoderStep/decoder/multi_rnn_cell/cell_0/cell_0/pointer_generator_attention_wrapper/pointer_generator_luong_attention/Squeeze:0", shape=(?, ?), dtype=float32) Tensor("decoder/while/PGDecoderStep/decoder/multi_rnn_cell/cell_0/cell_0/pointer_generator_attention_wrapper/cond/Merge:0", shape=(?, ?), dtype=float32)
Tensor("decoder/while/PGDecoderStep/decoder/multi_rnn_cell/cell_0/cell_0/pointer_generator_attention_wrapper/pointer_generator_luong_attention/Sum:0", shape=(?, ?), dtype=float32)
Tensor("decoder/while/PGDecoderStep/decoder/multi_rnn_cell/cell_0/cell_0/pointer_generator_attention_wrapper/pointer_generator_luong_attention_1/Squeeze:0", shape=(?, ?), dtype=float32) Tensor("decoder/while/PGDecoderStep/decoder/multi_rnn_cell/cell_0/cell_0/pointer_generator_attention_wrapper/cond_1/Merge:0", shape=(?, ?), dtype=float32)
Tensor("decoder/while/PGDecoderStep/decoder/multi_rnn_cell/cell_0/cell_0/pointer_g

In [16]:
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [17]:
def batching(X, Y):
    s_, es_, oovs_, target_ = [], [], [], []
    for x, y in zip(X, Y):
        s,es,oovs = sent2idx(x, dictionary_from)
        target = target2idx(y, oovs,dictionary_from)
        s_.append(s)
        es_.append(es)
        oovs_.append(oovs)
        target_.append(target)
    s_ = pad_sequences(s_,padding='post')
    es_ = pad_sequences(es_,padding='post')
    target_ = pad_sequences(target_,padding='post')
    maxlen = max([len(o) for o in oovs_])
    return s_, es_, target_, maxlen

In [18]:
from tqdm import tqdm
from sklearn.utils import shuffle
import time

for EPOCH in range(10):
    lasttime = time.time()
    total_loss, total_accuracy, total_loss_test, total_accuracy_test = 0, 0, 0, 0
    train_X, train_Y = shuffle(train_X, train_Y)
    test_X, test_Y = shuffle(test_X, test_Y)
    pbar = tqdm(range(0, len(train_X), batch_size), desc='train minibatch loop')
    for k in pbar:
        index = min(k+batch_size,len(train_X))
        batch_x, batch_es, batch_y, maxlen = batching(train_X[k: index],
                                                     train_Y[k: index])
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                                      feed_dict={model.X:batch_x,
                                                 model.source_extend_tokens:batch_es,
                                                model.Y:batch_y,
                                                model.source_oov_words:maxlen})
        total_loss += loss
        total_accuracy += acc
        pbar.set_postfix(cost=loss, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_X), batch_size), desc='test minibatch loop')
    for k in pbar:
        index = min(k+batch_size,len(test_X))
        batch_x, batch_es, batch_y, maxlen = batching(test_X[k: index],
                                                     test_Y[k: index])
        acc, loss = sess.run([model.accuracy, model.cost], 
                                      feed_dict={model.X:batch_x,
                                                 model.source_extend_tokens:batch_es,
                                                model.Y:batch_y,
                                                model.source_oov_words:maxlen})
        total_loss_test += loss
        total_accuracy_test += acc
        pbar.set_postfix(cost=loss, accuracy = acc)
        
    total_loss /= (len(train_X) / batch_size)
    total_accuracy /= (len(train_X) / batch_size)
    total_loss_test /= (len(test_X) / batch_size)
    total_accuracy_test /= (len(test_X) / batch_size)
        
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(EPOCH, total_loss, total_accuracy))
    print('epoch: %d, avg loss test: %f, avg accuracy test: %f'%(EPOCH, total_loss_test, total_accuracy_test))

train minibatch loop:   0%|          | 0/440 [00:00<?, ?it/s]

epoch: 0, avg loss: 64.281430, avg accuracy: 0.046412
epoch: 0, avg loss test: 51.371658, avg accuracy test: 0.020807


train minibatch loop:   0%|          | 0/440 [00:00<?, ?it/s]

epoch: 1, avg loss: 41.461005, avg accuracy: 0.021844
epoch: 1, avg loss test: 41.883894, avg accuracy test: 0.014441


train minibatch loop:   0%|          | 0/440 [00:00<?, ?it/s]

...

epoch: 7, avg loss: 8.189318, avg accuracy: 0.001758
epoch: 7, avg loss test: 40.533249, avg accuracy test: 0.004000


train minibatch loop:   0%|          | 0/440 [00:00<?, ?it/s]

epoch: 8, avg loss: 6.776059, avg accuracy: 0.001143
epoch: 8, avg loss test: 42.266799, avg accuracy test: 0.005249


test minibatch loop: 100%|██████████| 110/110 [00:29<00:00,  3.91it/s, accuracy=0, cost=31.4]     

epoch: 9, avg loss: 5.682627, avg accuracy: 0.001210
epoch: 9, avg loss test: 42.151334, avg accuracy test: 0.004053


In [19]:
batch_x, batch_es, batch_y, maxlen = batching(test_X[:1], test_Y[:1])

In [20]:
[rev_dictionary_from[i] for i in sess.run(model.beam_predictions, feed_dict = {model.X: batch_x,
                                             model.Y: batch_y,
                                             model.source_extend_tokens:batch_es,
                                             model.source_oov_words:maxlen})[0]][:7]

['dot', 'dot', 'dot', 'dot', 'dot', 'dot', 'dot']